In [1]:
file_path = r"C:\Users\Leo\Documents\code\parsiingpdftomysql\pdftoparsing.pdf"

In [3]:
#IMPORTS
import fitz
import os
from PIL import Image
import glob
import shutil
from pathlib import Path
import cv2



# Gerar as imagens com o fitz

In [4]:
doc = fitz.open(file_path)
for i in range(len(doc)):
    for img in doc.getPageImageList(i):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        if pix.n < 5:       # this is GRAY or RGB
            pix.writePNG("%s.png" % (xref))
        else:               # CMYK: convert to RGB first
            pix1 = fitz.Pixmap(fitz.csRGB, pix)
            pix1.writePNG("%s.png" % (xref))
            pix1 = None
        pix = None

Deprecation: 'getPageImageList' removed from class 'Document' after v1.19 - use 'get_page_images'.
Deprecation: 'writePNG' removed from class 'Pixmap' after v1.19 - use 'save'.


# Pegar todos os paths em uma lista

In [117]:
filesToExpurgue = (glob.glob(r"*.png")) 

# Deletar os que não tenhão resolução do RA ou da Foto

In [118]:
import cv2

path_dimension_height = []
path_dimension_width = []
test = []

for i in range(len(filesToExpurgue)):
    # read image
    img = cv2.imread(filesToExpurgue[i])
    test.append(filesToExpurgue[i])

    # get dimensions of image
    dimensions = img.shape

    # height, width, number of channels in image
    height = img.shape[0]
    width = img.shape[1]
    channels = img.shape[2]

    if height > 226 or width >226:
        os.remove(filesToExpurgue[i]) 


# Renomear para RA ou Foto

### Primeiro vamos definir a função que vai pegar os tamanhos

In [119]:
def getImageSize(image):
    im = Image.open(image)
    h, w = im.height, im.width
    im.close()
    result = h * w
    return result


### Depois vamos definir pegar uma lista das imagens usando o Glob

In [120]:
files = (glob.glob(r"*.png")) 

##### Separando as fotos dos QR Code

In [121]:
tipos = []
for i in range(len(files)):
    tipos.append(getImageSize(files[i]))
tipos

[14400, 51076, 14400, 51076, 14400, 51076]

#### colocando cada um na sua paginpasta

In [122]:
cwd = os.getcwd()
for i in range(len(tipos)):
    if tipos[i] == 14400:
        destination = cwd + "\\foto"
        shutil.move(files[i], destination)
    else:
        destination = cwd + "\\qrcode"
        shutil.move(files[i], destination)

#### convertendo para base 64

In [4]:
import base64
from PIL import Image
from io import BytesIO

with open(r"C:\Users\Leo\Documents\code\parsiingpdftomysql\foto\6.png", "rb") as image_file:
    data = base64.b64encode(image_file.read())

print (data)


b'...'


In [ ]:
def recuperandoImagemBase64(data):
    im = Image.open(BytesIO(base64.b64decode(data)))
    im.save('image1.png', 'PNG')